# Load the tests to compare

In [1]:
import os
import sys
import json
import pickle
import pandas as pd
__file__ = globals()['_dh'][0]

# get an absolute path to the directory that contains parent files
project_dir = __file__ = globals()['_dh'][0]
sys.path.append(os.path.normpath(os.path.join(project_dir, '..', '..')))

from experiments.utils.constants import TESTS_RESULTS_PATH
series = 61
start_test = 0
end_test = 10
tests_ids = [i for i in range(start_test, end_test)]
tests_names = [i for i in range(start_test, end_test)]

tests = {}


for test_id, test_name in zip(tests_ids, tests_names):
    test_path = os.path.join(
        TESTS_RESULTS_PATH,
        'series',
        str(series),
        'tests',
        str(test_id))
    with open(os.path.join(test_path, 'info.json')) as cf:
        info = json.loads(cf.read())
    with open(os.path.join(test_path, 'episodes.pickle'), 'rb') as input_file:
        episodes = pickle.load(input_file)
    tests.update({test_name: {
        'info': info,
        'episodes': episodes
    }})
test_types = [test['info']['algorithm'] for _, test in tests.items()]
print("availeble tests:\n {}\ntests_types:\n {}".format(
    list(tests.keys()),
    test_types))

availeble tests:
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
tests_types:
 ['PPO', 'PPO', 'PPO', 'PPO', 'PPO', 'PPO', 'PPO', 'PPO', 'PPO', 'PPO']


# Necessary stats of a test

In [2]:
final_stats = []
for test_id in range(start_test, end_test):
    users_distances_episodes_average = [episode['users_distances'].mean() for episode in tests[test_id]['episodes']]
    users_distances_test_average = sum(users_distances_episodes_average) / len(users_distances_episodes_average)
    num_consolidated_episodes_average = [episode['num_consolidated'].mean() for episode in tests[test_id]['episodes']]
    num_consolidated_test_average = sum(num_consolidated_episodes_average) / len(num_consolidated_episodes_average)
    # print(users_distances_episodes_average)
    # print(users_distances_test_average)
    # print(num_consolidated_episodes_average)
    # print(num_consolidated_test_average)
    final_stats.append({
        'test_id': test_id,
        'checkpoint': tests[test_id]['info']['checkpoint'],
        'network_id': tests[test_id]['info']['network_id'],
        'trace_id_test': tests[test_id]['info']['trace_id_test'],
        'experiments': tests[test_id]['info']['experiments'],
        'penalty_latency': tests[test_id]['info']['penalty_latency'],
        'penalalty_consolidated': tests[test_id]['info']['penalty_consolidated'],
        'users_distances_test_average': users_distances_test_average,
        'num_consolidated_test_average': num_consolidated_test_average
    })

final_stats_df = pd.DataFrame(final_stats)
final_stats_df['test_types'] = test_types
# final_stats_df.loc[df['penalalty_consolidated']]

# TODO add the following information to the table (from the info.json file)
# - penatly_latency
# - penalty_conslidation

In [3]:
tests[test_id]['info']

{'series': 61,
 'dataset_id': 7,
 'workload_id': 0,
 'network_id': 3,
 'trace_id': 2,
 'trace_id_test': 2,
 'checkpoint': 1000,
 'experiment_str': 'PPO_SimEdgeEnv_1e79e_00000_0_2021-12-09_23-17-03',
 'experiments': 4,
 'episode_length': 10,
 'num_episodes': 5,
 'algorithm': 'PPO',
 'penalty_latency': 0,
 'penalty_consolidated': 10}

# compare tests - tables

In [4]:
final_stats_df

,test_id,checkpoint,network_id,trace_id_test,experiments,penalty_latency,penalalty_consolidated,users_distances_test_average,num_consolidated_test_average,test_types
0,0,1000,3,0,0,10,0,1.469588,1.24,PPO
1,1,1000,3,0,1,10,5,1.426340,2.80,PPO
2,2,1000,3,0,2,10,10,1.549941,1.86,PPO
3,3,1000,3,0,3,2,10,1.462613,2.92,PPO
4,4,1000,3,0,4,0,10,1.393866,0.82,PPO
5,5,1000,3,2,0,10,0,1.221664,1.62,PPO
6,6,1000,3,2,1,10,5,1.277163,2.98,PPO
7,7,1000,3,2,2,10,10,1.266141,2.86,PPO
8,8,1000,3,2,3,2,10,1.315529,2.98,PPO
9,9,1000,3,2,4,0,10,1.391491,0.68,PPO


In [5]:
final_stats_df[(final_stats_df['experiments'] == 0) & (final_stats_df['trace_id_test'] == 2)].sort_values(by=['num_consolidated_test_average'])

,test_id,checkpoint,network_id,trace_id_test,experiments,penalty_latency,penalalty_consolidated,users_distances_test_average,num_consolidated_test_average,test_types
5,5,1000,3,2,0,10,0,1.221664,1.62,PPO


# compare tests - plots

In [6]:
final_stats_df[(final_stats_df['experiments'] == 4) & (final_stats_df['trace_id_test'] == 2)].sort_values(by=['users_distances_test_average'])

,test_id,checkpoint,network_id,trace_id_test,experiments,penalty_latency,penalalty_consolidated,users_distances_test_average,num_consolidated_test_average,test_types
9,9,1000,3,2,4,0,10,1.391491,0.68,PPO


In [7]:
final_stats_df[(final_stats_df['network_id'] == 7) & (final_stats_df['trace_id_test'] == 2)].sort_values(by=['users_distances_test_average']) #& (


,test_id,checkpoint,network_id,trace_id_test,experiments,penalty_latency,penalalty_consolidated,users_distances_test_average,num_consolidated_test_average,test_types
